In [1]:
import os
import sys

path = os.getcwd()
while os.path.basename(os.path.normpath(path)) != 'mejiro':
    path = os.path.dirname(path)
repo_path = path
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from glob import glob
import multiprocessing
import pandas as pd
from tqdm import tqdm
import time
from copy import deepcopy
from multiprocessing import Pool
from hydra import initialize, compose
import pickle
import pandas as pd

from mejiro.plots import diagnostic_plot, plot
from mejiro.utils import util
from mejiro.helpers import pyhalo

In [6]:
def add(tuple):
    from mejiro.helpers import pyhalo
    from mejiro.utils import util

    # unpack tuple
    (lens, pipeline_params, output_dir) = tuple

    # unpack pipeline_params
    subhalo_cone = pipeline_params['subhalo_cone']
    los_normalization = pipeline_params['los_normalization']
   
    z_lens = round(list(lens.values())[0].z_lens, 2)
    z_source = round(list(lens.values())[0].z_source, 2)

    # randomly generate CDM subhalos
    halo_tuple = pyhalo.generate_CDM_halos(z_lens, z_source, cone_opening_angle_arcsec=subhalo_cone, LOS_normalization=los_normalization)

    # print(halo_tuple)

    # add this subhalo population to the lens for each filter
    for band, band_lens in lens.items():
        band_lens.add_subhalos(*halo_tuple)

        pickle_target = os.path.join(output_dir, f'lens_with_subhalos_{band_lens.uid}_{band}')
        util.pickle(pickle_target, band_lens)

In [7]:
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, repo_dir, pickle_dir = config.machine.array_dir, config.machine.data_dir, config.machine.repo_dir, config.machine.pickle_dir
util.create_directory_if_not_exists(pickle_dir)

In [8]:
# split up the lenses into batches based on core count
cpu_count = multiprocessing.cpu_count()
process_count = cpu_count - 4

# get bands
bands = util.hydra_to_dict(config.pipeline)['band']

# organize the pickles into a dict
lens_dict = {}
for band in bands:
    # open pickled lens list
    lens_list = util.unpickle(os.path.join(pickle_dir, f'01_skypy_output_lens_list_{band.lower()}'))
    lens_dict[band] = lens_list

# TODO this naming is dumb, fix it
lenses = []
# create a tuple for each lens
for i, _ in enumerate(lens_list):
    lens = {}
    for band in bands:
        lens[band] = lens_dict[band][i]
    lenses.append(lens)

# directory to write the lenses with subhalos to
output_dir = os.path.join(pickle_dir, '02_lenses_with_substructure')
util.create_directory_if_not_exists(output_dir)
util.clear_directory(output_dir)

# tuple the parameters
pipeline_params = util.hydra_to_dict(config.pipeline)
tuple_list = []
for i, _ in enumerate(lenses):
    tuple_list.append((lenses[i], pipeline_params, output_dir))

# batch
generator = util.batch_list(tuple_list, process_count)
batches = list(generator)

# process the batches
for batch in tqdm(batches):
    pool = Pool(processes=process_count)
    pool.map(add, batch)

  0%|          | 0/79 [00:50<?, ?it/s]


AttributeError: 'dict' object has no attribute 'uid'